In [19]:
import os 
from langchain_openai import ChatOpenAI

from langchain.chains import RetrievalQA
from langchain.schema.runnable import RunnableMap
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.agents import AgentExecutor, initialize_agent, AgentType
from langchain.agents.agent_toolkits import GmailToolkit
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_community.document_loaders.excel import UnstructuredExcelLoader

## 1. Create a model

In [20]:
# from langchain.llms import Ollama
# # llm = Ollama(base_url = "http://localhost:11434",model = "llama2")

In [21]:
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
gmail_toolkit = GmailToolkit()
llm=ChatOpenAI(temperature=0.0,model="gpt-3.5-turbo")


### 2. Load the documents and split it

In [22]:
# from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
# import google.generativeai as genai
# from langchain_google_genai import ChatGoogleGenerativeAI

# genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
# generation_config = {
#     "temperature": 0.9,
#     "top_k": 2,
# }
# llm = ChatGoogleGenerativeAI(model = "gemini-pro", generation_config =generation_config)


In [23]:

# Load the document
file_path = "info.xlsx"
loader = UnstructuredExcelLoader(file_path)
docs = loader.load()
# Split the data
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
splits = text_splitter.split_documents(docs)

# embeddings = OpenAIEmbeddings(model = 'text-embedding-3-large')
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# To database

db = Chroma.from_documents(documents= splits, embedding = embeddings)
retreiver = db.as_retriever()

## 3. Create embeddings and retriever

### 4. Retrieving mails from ajeetacharya02@gmail.comm

In [24]:
output = {"emails":{
    "sender": "john.doe@example.com",
    "subject": "delivery charge?",
    "content": "What is the delivery charge?",
  }
}
type(output)

dict

In [25]:
def check_for_mail():
    prompt =f"""You have access to an external system that can check email. Please use it to see if I have received a new email. If there are any, filter out non-essential ones such as newsletters, Blogs, promotional content and notifications.Use your expertise in email content analysis to distinguish emails from the rest, pay attention to the sender and avoid invalid emails. Return the response as a DICT containing the sender's email address,the subject line of each email and the contents in the email,  separated by commas without double inverted commas in the output."""

    agent = initialize_agent(agent= AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                            tools= gmail_toolkit.get_tools(), 
                            llm= llm, 
                            verbose = True, 
                            max_execution_time = 1600,
                            max_iterations = 1000)
    gmail_toolkit.mark_as_read(os.getenv('MY_EMAIL'))
    # prompt = PromptTemplate(template=template)
    response = agent.run(prompt)
    return response

In [26]:
response = check_for_mail()
response

AttributeError: 'GmailToolkit' object has no attribute 'mark_as_read'

In [ ]:
type(response)

dict

NameError: name 'emails' is not defined

In [ ]:
prompt = """
You are a virtual assistant who answers users query. When a user asks a question  
you have to answer him based on the context given in three back ticks and make sure to provide each and every details.
context: ```{context}```
question is given in double back ticks : ``{question}``.
Provide a good answer that is related to context and DONOT EXCEED MORE THAN 30 WORDS.
If the question is out of context, do not provide the wrong answer just say i am sorry i don't have an answer to that question. Please contact info@palmminds.com or call to +977 9123123 for more info

"""

prompt = """

You are a virtual assistant who answers users query. When a user asks a question  
you have to answer him based on the context given in three back ticks and make sure to provide each and every details.
context: ```{context}```
question is given in double back ticks : ``{question}``.

If the question is out of context, do not provide the wrong answer just say i am sorry i don't have an ansawer to that question.

"""

In [ ]:
template = ChatPromptTemplate.from_template(prompt)
output_parser = StrOutputParser()
chain = RunnableMap(
    {
        "context": lambda x:db.similarity_search(x['question'], k =1),
        "question": lambda x:x['question']
    }
)|template|llm|output_parser


In [27]:
response

{'emails': [{'sender': 'roshnithapaa@gmail.com',
   'subject': 'CHITCHAT',
   'content': 'Hello, how can i place my order?'},
  {'sender': 'roshnithapaa@gmail.com',
   'subject': 'place',
   'content': 'i want to place order.'},
  {'sender': 'roshnithapaa@gmail.com',
   'subject': 'FLIGHT',
   'content': 'I want to deliver myself to japan'},
  {'sender': 'roshnithapaa@gmail.com',
   'subject': 'flight',
   'content': 'I want to go to japan'}]}

In [15]:
email_data = response['emails']
email_data

[{'sender': 'roshnithapaa@gmail.com',
  'subject': 'CHITCHAT',
  'content': 'Hello, how can i place my order?'},
 {'sender': 'roshnithapaa@gmail.com',
  'subject': 'place',
  'content': 'i want to place order.'},
 {'sender': 'roshnithapaa@gmail.com',
  'subject': 'FLIGHT',
  'content': 'I want to deliver myself to japan'},
 {'sender': 'roshnithapaa@gmail.com',
  'subject': 'flight',
  'content': 'I want to go to japan'}]

In [ ]:
to_send = email_data[1]

{'sender': 'roshnithapaa@gmail.com',
 'subject': 'place',
 'content': 'i want to place order.'}

In [ ]:
import os, smtplib
import email.message as em


def send_email(to_email, subject, message):
    smtp_server = os.getenv('SMTP_HOST')
    smtp_port = int(os.getenv('SMTP_PORT'))  # Convert to int
    sender_email = os.getenv('EMAIL_DEV')
    password = os.getenv('EMAIL_PASSWORD')

    msg = em.Message()
    msg['From'] = sender_email
    msg['To'] = to_email
    msg['Subject'] = subject
    msg.add_header('Content-Type', 'text/html')
    msg.set_payload(message)

    smtp = smtplib.SMTP(smtp_server, smtp_port)
    smtp.starttls()
    smtp.login(sender_email, password)
    smtp.sendmail(msg['From'], [msg['To']], msg.as_string())
    smtp.quit()
    print("Successfully sent email to", to_email)


In [16]:
for email in email_data:
    print(email['content'])
    result = chain.invoke({"question":email['content']})
    send_email(to_email=email['sender'], subject=email['subject'], message=result)
    print(result)

Hello, how can i place my order?
I am sorry, I don't have an answer to that question. Please contact info@palmminds.com or call +977 9123123 for more info.
i want to place order.
```i want to place order.```
To place an order, visit our website and select the items you wish to purchase. Add them to your cart and proceed to checkout.
I want to deliver myself to japan
I am sorry I don't have an answer to that question. Please contact info@palmminds.com or call +977 9123123 for more info.
I want to go to japan
I am sorry I don't have an answer to that question. Please contact info@palmminds.com or call +977 9123123 for more info.


In [ ]:
import os, smtplib
import email.message 


def send_email(to_email, subject, message):
    smtp_server = os.getenv('SMTP_HOST')
    smtp_port = int(os.getenv('SMTP_PORT'))  # Convert to int
    sender_email = os.getenv('EMAIL_DEV')
    password = os.getenv('EMAIL_PASSWORD')

    msg = email.message.Message()
    msg['From'] = sender_email
    msg['To'] = to_email
    msg['Subject'] = subject
    msg.add_header('Content-Type', 'text/html')
    msg.set_payload(message)

    smtp = smtplib.SMTP(smtp_server, smtp_port)
    smtp.starttls()
    smtp.login(sender_email, password)
    smtp.sendmail(msg['From'], [msg['To']], msg.as_string())
    smtp.quit()
    print("Successfully sent email to", to_email)
